### 11.9 Transformer
論文データを学習させてナンチャッテ論文をlatex形式で出力させる<br>
Transformenr版

### ライブラリインポート

In [1]:
!pip install datasets tqdm torch requests feedparser --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.8 MB/s eta 0:00:00


In [2]:
import os
import tarfile
import glob
import re
import requests
from tqdm import tqdm
import urllib.parse
import feedparser
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

### データセット読み込み

#### arXiv API から論文 ID をまとめて取得する関数

In [3]:
def fetch_arxiv_ids(category="cs.LG", max_results=200):
    """
    指定カテゴリの新しい arXiv 論文 ID をまとめて取得する簡易関数
    例: category="cs.LG"（Machine Learning）
    """
    base_url = "http://export.arxiv.org/api/query"

    # APIのクエリ: カテゴリ指定 + 新しい順
    search_query = f"cat:{category}"
    params = {
        "search_query": search_query,
        "start": 0,
        "max_results": max_results,
        "sortBy": "submittedDate",
        "sortOrder": "descending",
    }

    url = base_url + "?" + urllib.parse.urlencode(params)
    print("🔍 arXiv API URL:", url)

    feed = feedparser.parse(url)

    paper_ids = []
    for entry in feed.entries:
        # entry.id 例: "http://arxiv.org/abs/2401.12345v1"
        m = re.search(r'arxiv.org/abs/(\d{4}\.\d+)', entry.id)
        if m:
            pid = m.group(1)  # "2401.12345"
            paper_ids.append(pid)

    print(f"📄 取得した論文ID数: {len(paper_ids)} 件")
    return paper_ids

# ここで好きなカテゴリ・件数を指定
# 例1: cs.LG (Machine Learning) 最新から200本
# 例2: math.PR (Probability)
paper_ids = fetch_arxiv_ids(category="cs.LG", max_results=200)

print(paper_ids)

🔍 arXiv API URL: http://export.arxiv.org/api/query?search_query=cat%3Acs.LG&start=0&max_results=200&sortBy=submittedDate&sortOrder=descending
📄 取得した論文ID数: 200 件
['2511.21690', '2511.21689', '2511.21686', '2511.21678', '2511.21675', '2511.21669', '2511.21668', '2511.21667', '2511.21654', '2511.21652', '2511.21638', '2511.21635', '2511.21626', '2511.21622', '2511.21613', '2511.21607', '2511.21600', '2511.21594', '2511.21590', '2511.21581', '2511.21566', '2511.21561', '2511.21560', '2511.21550', '2511.21537', '2511.21531', '2511.21526', '2511.21514', '2511.21513', '2511.21500', '2511.21490', '2511.21474', '2511.21466', '2511.21465', '2511.21437', '2511.21416', '2511.21414', '2511.21408', '2511.21397', '2511.21381', '2511.21378', '2511.21377', '2511.21369', '2511.21364', '2511.21363', '2511.21356', '2511.21354', '2511.21350', '2511.21340', '2511.21338', '2511.21335', '2511.21320', '2511.21283', '2511.21276', '2511.21257', '2511.21247', '2511.21232', '2511.21223', '2511.21215', '2511.21213'

#### arXiv ソースをダウンロード＆展開

In [4]:
os.makedirs("papers", exist_ok=True)

for pid in tqdm(paper_ids, desc="Downloading", ncols=80, ascii=True):
    url = f"https://arxiv.org/src/{pid}v1"
    out_path = f"papers/{pid}.tar.gz"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(r.content)
            # tar / tar.gz / その他の圧縮形式に柔軟に対応
            try:
                with tarfile.open(out_path, "r:*") as tar:
                    tar.extractall(f"papers/{pid}")
            except tarfile.ReadError:
                print(f"Skipped {pid} (not a tar file)")
        else:
            print(f"Skipped {pid} (status {r.status_code})")
    except Exception as e:
        print(f"Error downloading {pid}: {e}")


Downloading:   0%|                                      | 0/200 [00:00<?, ?it/s]/tmp/ipython-input-2326405988.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(f"papers/{pid}")
Downloading:  12%|###3                         | 23/200 [00:10<00:36,  4.79it/s]

Skipped 2511.21561 (not a tar file)


Downloading:  38%|##########8                  | 75/200 [00:31<00:22,  5.44it/s]

Skipped 2511.21089 (not a tar file)


Downloading:  44%|############9                | 89/200 [00:43<02:52,  1.55s/it]

...

Downloading:  60%|################6           | 119/200 [00:58<00:27,  2.90it/s]

Skipped 2511.20871 (not a tar file)


Downloading:  94%|##########################1 | 187/200 [02:12<00:05,  2.50it/s]

Skipped 2511.20382 (not a tar file)


Downloading: 100%|############################| 200/200 [02:18<00:00,  1.44it/s]


#### 結果確認

In [6]:
!find papers -name "*.tex" | head -n 20

papers/2511.20406/imports.tex
papers/2511.20406/log_2025.tex
papers/2511.21550/content/intro.tex
papers/2511.21550/content/heavy_momentum_mamba.tex
papers/2511.21550/content/related_work.tex
papers/2511.21550/content/_abstract.tex
papers/2511.21550/content/result.tex
papers/2511.21550/content/abstract.tex
papers/2511.21550/content/har_system.tex
papers/2511.21550/content/vanishing_gradient.tex
papers/2511.21550/content/intro_.tex
papers/2511.21550/content/prelim.tex
papers/2511.21550/content/conclusion.tex
papers/2511.21550/content/beyond.tex
papers/2511.21550/content/har_system_.tex
papers/2511.21550/content/prelim_.tex
papers/2511.21550/content/continuous_momentum.tex
papers/2511.21550/New_IEEEtran_how-to.tex
papers/2511.21550/bare_jrnl_new_sample4.tex
papers/2511.21550/math_commands.tex


### データ加工

#### LaTeX 本文抽出 ＋ 前処理

In [8]:
import re, glob

def extract_latex_body(tex_text: str) -> str:
    """LaTeX本文を抽出して、数式を取り除きつつクリーンアップする（簡易版）"""
    # コメント行を削除（% 以降）
    text = re.sub(r'%.*', '', tex_text)

    # \begin{document}〜\end{document} の間だけを抜き出し
    m = re.search(r'\\begin{document}(.*?)\\end{document}', text, re.DOTALL)
    if m:
        text = m.group(1)

    # bibliography / 参考文献・付録以降をざっくり削る
    text = re.split(r'\\bibliography|\\begin{thebibliography}', text)[0]
    text = re.split(r'\\appendix', text)[0]

    # ==== ここから数式削除ブロック ====

    # ディスプレイ数式 $$ ... $$
    text = re.sub(r'\$\$(.+?)\$\$', ' ', text, flags=re.DOTALL)

    # ディスプレイ数式 \[ ... \]
    text = re.sub(r'\\\[(.+?)\\\]', ' ', text, flags=re.DOTALL)

    # インライン数式 $ ... $
    text = re.sub(r'\$(.+?)\$', ' ', text, flags=re.DOTALL)

    # equation / align / gather / multline 系の環境を削除
    math_envs = [
        "equation", "equation*",
        "align", "align*",
        "gather", "gather*",
        "multline", "multline*",
    ]
    for env in math_envs:
        pattern = rf'\\begin{{{env}}}(.+?)\\end{{{env}}}'
        text = re.sub(pattern, ' ', text, flags=re.DOTALL)

    # ==== 数式削除ここまで ====

    # 連続する空白を1つにまとめる
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [9]:
# すべての .tex から本文を抽出（数式は除去済み）
all_texts = []
for fname in glob.glob("papers/**/*.tex", recursive=True):
    try:
        with open(fname, encoding="utf-8", errors="ignore") as f:
            raw = f.read()
        cleaned = extract_latex_body(raw)
        if len(cleaned) > 200:  # ごく短いファイルはノイズなので除外
            all_texts.append(cleaned)
            # print(f"extracted from: {fname}")
    except Exception as e:
        print(f"{fname}: {e}")

print(f"\n抽出された論文本数: {len(all_texts)}")
print(f"総テキスト長: {sum(len(t) for t in all_texts):,} 文字")

# テキストを一つにまとめたファイルも保存しておく（デバッグ用）
merged_text = "\n\n".join(all_texts)
with open("merged_corpus.tex", "w", encoding="utf-8") as f:
    f.write(merged_text)
print("\nmerged_corpus.tex に保存しました")



抽出された論文本数: 860
総テキスト長: 9,338,060 文字

merged_corpus.tex に保存しました


#### トークン化と語彙辞書作成

In [22]:
# ---- トークナイズと語彙構築 ----
from collections import Counter

# トークナイズパターン
# （LaTeXコマンドや記号も一応トークンにしていますが、
#  数式本体は上の extract_latex_body で削除済み）
TOKEN_PATTERN = r"[A-Za-z]+|[.,;:!?()]"
SPECIAL_TOKENS = ["<unk>", "<bos>", "<eos>"]

tokens = []
for txt in all_texts:
    # 各論文ごとに <bos>, <eos> を挟んでトークナイズ
    body_tokens = re.findall(TOKEN_PATTERN, txt)
    tokens.extend(["<bos>"] + body_tokens + ["<eos>"])

print(f"トークン総数: {len(tokens):,}")

# 語彙を頻度上位だけに圧縮
counter = Counter(tokens)

VOCAB_LIMIT = 10000
most_common_tokens = [
    w for (w, c) in counter.most_common(VOCAB_LIMIT)
    if w not in SPECIAL_TOKENS
]

vocab = SPECIAL_TOKENS + most_common_tokens

word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

UNK_IDX = word2idx["<unk>"]
BOS_IDX = word2idx["<bos>"]
EOS_IDX = word2idx["<eos>"]

print(f"語彙数（圧縮後）: {len(vocab):,}")

# トークン列を index に変換
indexed_tokens = [word2idx.get(t, UNK_IDX) for t in tokens]

# 念のためチェック
max_id = max(indexed_tokens)
print("最大トークンID:", max_id)
print("語彙サイズ:", len(vocab))
assert max_id < len(vocab), "トークンIDが語彙サイズを超えています！"


トークン総数: 1,548,730
語彙数（圧縮後）: 10,001
最大トークンID: 10000
語彙サイズ: 10001


### データセット定義

#### Datasetクラス定義

In [23]:
class LatexDataset(Dataset):
    """大きなトークン列から、ランダムにサンプルを切り出す Dataset"""
    def __init__(self, data, seq_length=60, samples_per_epoch=3000):
        self.data = data                      # ← すでに index 化された列
        self.seq_length = seq_length
        self.samples_per_epoch = samples_per_epoch

        self.max_start = len(self.data) - (self.seq_length + 1)
        assert self.max_start > 0, "データが短すぎます"

    def __len__(self):
        # 1epochあたりのサンプル数を直接決める
        return self.samples_per_epoch

    def __getitem__(self, idx):
        # idx は使わず、毎回ランダムな位置から切り出す
        start = random.randint(0, self.max_start)
        end = start + self.seq_length + 1
        chunk = self.data[start:end]

        x = torch.tensor(chunk[:-1], dtype=torch.long)  # [T]
        y = torch.tensor(chunk[1:], dtype=torch.long)   # [T]
        return x, y


In [24]:
# ここで初めて dataset を作る（tokens ではなく indexed_tokens を渡す）
seq_length = 128
samples_per_epoch = 8000
dataset = LatexDataset(indexed_tokens, seq_length=seq_length,
                       samples_per_epoch=samples_per_epoch)


#### Train / Validation に分割して DataLoader を作成

In [25]:
train_ratio = 0.9
n_total = len(dataset)
n_train = int(n_total * train_ratio)
n_val = n_total - n_train

train_ds, val_ds = random_split(dataset, [n_train, n_val])

batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=2,
    pin_memory=True
)

print(f"データ数: total={n_total}, train={len(train_ds)}, val={len(val_ds)}")
print(f"1epoch のステップ数（train）: {len(train_loader)}")


データ数: total=8000, train=7200, val=800
1epoch のステップ数（train）: 225


### 学習

#### Transformerモデル定義(疑似論文生成)

In [26]:
# Transformerモデル定義(疑似論文生成)
class LatexTransformer(nn.Module):
    """
    LaTeX トークン列用のシンプルな Transformer 言語モデル
    入力: x [B, T]  (トークンID)
    出力: logits [B, T, V]  (各位置ごとの単語分布のロジット)
    """
    def __init__(self,
                 vocab_size: int,
                 d_model: int = 384,
                 nhead: int = 6,
                 num_layers: int = 6,
                 dim_feedforward: int = 1024,
                 dropout: float = 0.1,
                 max_seq_len: int = 512):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        # 語彙埋め込み
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        # 位置埋め込み（シンプルな learnable embedding）
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True  # [B, T, E] で処理
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        # 出力層
        self.fc_out = nn.Linear(d_model, vocab_size)
        # Causal mask をキャッシュ用に保持
        self.register_buffer("mask", None, persistent=False)

    def _generate_square_subsequent_mask(self, sz: int, device: torch.device):
        """
        自己回帰用のマスク (未来のトークンを見ないようにする)
        shape: [T, T]
        """
        if (self.mask is None) or (self.mask.size(0) != sz):
            mask = torch.full((sz, sz), float("-inf"), device=device)
            mask = torch.triu(mask, diagonal=1)
            self.mask = mask
        return self.mask

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [B, T]  トークンID列
        戻り値:
            logits: [B, T, V]
        """
        B, T = x.size()
        device = x.device
        # 位置ID [0, 1, ..., T-1]
        positions = torch.arange(T, device=device).unsqueeze(0).expand(B, T)  # [B, T]
        tok_emb = self.token_embedding(x)          # [B, T, d_model]
        pos_emb = self.pos_embedding(positions)    # [B, T, d_model]
        h = tok_emb + pos_emb                      # [B, T, d_model]
        # causal mask (未来の情報を見ない)
        src_mask = self._generate_square_subsequent_mask(T, device=device)  # [T, T]
        # TransformerEncoder に通す
        # batch_first=True なので h: [B, T, d_model], mask は [T, T]
        h = self.transformer(h, mask=src_mask)     # [B, T, d_model]
        logits = self.fc_out(h)                    # [B, T, V]
        return logits


#### 繰り返し計算

In [28]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

vocab_size = len(vocab)
model = LatexTransformer(
    vocab_size=vocab_size,
    d_model=256,          # 384 → 256
    nhead=4,              # 6 → 4
    num_layers=4,         # 6 → 4
    dim_feedforward=768,  # 1024 → 768
    dropout=0.1,
    max_seq_len=512
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

num_epochs = 30
max_grad_norm = 1.0  # 勾配クリッピングの最大ノルム

for epoch in range(num_epochs):
    # -------------------------------
    # Train
    # -------------------------------
    model.train()
    total_train_loss = 0.0

    for x, y in train_loader:
        # x, y: [B, T]
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)  # [B, T, V]

        # CrossEntropyLoss は [N, C] vs [N] をとるので reshape する
        loss = criterion(
            logits.view(-1, logits.size(-1)),  # [B*T, V]
            y.view(-1)                         # [B*T]
        )
        loss.backward()

        # 勾配クリッピング（勾配爆発対策）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # -------------------------------
    # Validation
    # -------------------------------
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            val_loss = criterion(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # 学習率スケジューラを進める
    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"train_loss={avg_train_loss:.4f}  val_loss={avg_val_loss:.4f}  "
        f"lr={scheduler.get_last_lr()[0]:.5f}"
    )


device: cuda
Epoch [1/30] train_loss=6.4095  val_loss=5.6767  lr=0.00100
Epoch [2/30] train_loss=5.3215  val_loss=5.0340  lr=0.00100
Epoch [3/30] train_loss=4.8536  val_loss=4.6585  lr=0.00100
Epoch [4/30] train_loss=4.5790  val_loss=4.3923  lr=0.00100
Epoch [5/30] train_loss=4.3778  val_loss=4.2029  lr=0.00100
Epoch [6/30] train_loss=4.2348  val_loss=4.0388  lr=0.00100
Epoch [7/30] train_loss=4.0965  val_loss=4.0069  lr=0.00100
Epoch [8/30] train_loss=3.9862  val_loss=3.8245  lr=0.00100
Epoch [9/30] train_loss=3.8949  val_loss=3.7119  lr=0.00100
Epoch [10/30] train_loss=3.7974  val_loss=3.6445  lr=0.00050
Epoch [11/30] train_loss=3.6699  val_loss=3.4911  lr=0.00050
Epoch [12/30] train_loss=3.5991  val_loss=3.4043  lr=0.00050
Epoch [13/30] train_loss=3.5681  val_loss=3.4186  lr=0.00050
Epoch [14/30] train_loss=3.5109  val_loss=3.3464  lr=0.00050
Epoch [15/30] train_loss=3.4644  val_loss=3.2962  lr=0.00050
Epoch [16/30] train_loss=3.4482  val_loss=3.2757  lr=0.00050
Epoch [17/30] train_

### 予測

#### サンプリング用ヘルパー




In [29]:
def sample_from_logits(logits: torch.Tensor,
                       temperature: float = 0.8,
                       top_k: int = 50) -> int:
    """
    logits: [V] の1時刻分のロジット
    戻り値: 次に選ぶトークンID
    """
    # 温度でスケーリング
    logits = logits / temperature

    # top-k だけ残してサンプリング（極端なノイズを減らす）
    if top_k is not None and top_k > 0:
        values, indices = torch.topk(logits, top_k)
        probs = torch.softmax(values, dim=-1)
        idx_in_topk = torch.multinomial(probs, 1).item()
        next_token_id = indices[idx_in_topk].item()
        return next_token_id
    else:
        probs = torch.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, 1).item()
        return next_token_id


#### テキスト生成関数

In [37]:
def generate(seed_text, length=200, temperature=0.8, top_k=50,
             seq_length=60, repetition_penalty=1.2, penalty_window=50,
             math_penalty=3.0):

    model.eval()
    words = re.findall(TOKEN_PATTERN, seed_text)

    # 数式関連ワード（抑制対象）
    MATH_TOKENS = ["begin", "align", "frac", "theta", "exp", "sum", "geq",
               "cdot", "Bigg", "Pr", "Var", "sim", "=","_","^",
               "ref", "eqref", "cite", "citet", "apdx", "section", "Fig"]

    for _ in range(length):
        seq = [word2idx.get(w, UNK_IDX) for w in words[-seq_length:]]
        x = torch.tensor([seq], dtype=torch.long).to(device)

        with torch.no_grad():
            logits = model(x)[0, -1]

            logits[UNK_IDX] -= 10.0  # <unk> 減少

            # 🔥 数式トークンを抑制
            for tk in MATH_TOKENS:
                if tk in word2idx:
                    logits[word2idx[tk]] -= math_penalty

            # 🔁 繰り返し抑制
            recent = seq[-penalty_window:]
            for tid in set(recent):
                logits[tid] /= repetition_penalty

            logits = logits / temperature

            values, indices = torch.topk(logits, top_k)
            probs = torch.softmax(values, dim=0)
            next_idx = indices[torch.multinomial(probs, 1).item()].item()

        words.append(idx2word[next_idx])

    return " ".join(words)


#### テキスト生成サンプル

In [42]:
seed = (
    "In this paper, we explore a novel approach to improving model "
    "robustness under changing data distributions. "
    "Our method is motivated by the observation that"
)
print("\n生成結果:\n")
text = generate(
    seed_text=seed,
    length=280,
    temperature=0.6,
    top_k=15,
    repetition_penalty=1.3,
    math_penalty=6.0
)

print(text)


生成結果:

In this paper , we explore a novel approach to improving model robustness under changing data distributions . Our method is motivated by the observation that effectively prioritizes training and inference time constraints . The proposed method can be viewed as a framework for unsupervised anomaly detection . By leveraging GMM adapts ensemble performance in terms of ranking tasks , we demonstrate why this approach is robust to its effectiveness on improving model robustness by explicitly aligning the reconstruction quality and overall accuracy with improved robustness while retaining both nDCG and nDCG capacity . This suggests that our method effectively prevents interpretability from cross modal feature retention under extreme imbalance , where models exhibit stronger discrimination ( Figure ref fig : intro toy experiments results for further ablation study ) to reduce the number of modalities . subsection Ablation Study label sec : ablations The effectiveness of this work is t

####  生成AI用プロンプト
この生成された英文を、論文の「Related Work」として自然な英語に整形してください。 LaTeX 構文（section, cite など）は削除し、文法の不自然な部分を修正し、 意味が通るまとまった段落（100〜150 words）にしてください。

In [44]:
formatted_related_work = (
    "Our work introduces a novel approach to enhancing model robustness against evolving data distributions, "
    "designed to optimize both training and inference efficiency. This method effectively addresses "
    "interpretability challenges stemming from cross-modal feature retention, particularly in scenarios "
    "of extreme data imbalance. While prior research has investigated methods for tackling long-tailed "
    "imbalances in multimodal learning, leveraging techniques such as LMPT and deep generative modeling, "
    "these often depend on hand-engineered features or struggle with robust representation learning. We "
    "show that our approach not only consistently improves robustness and generalization capabilities "
    "through a unified modality fusion mechanism but also performs well in unsupervised anomaly "
    "detection, distinguishing itself from existing models that frequently suffer from noise "
    "sensitivity and struggle to maintain performance across varied datasets."
)
print(formatted_related_work)

Our work introduces a novel approach to enhancing model robustness against evolving data distributions, designed to optimize both training and inference efficiency. This method effectively addresses interpretability challenges stemming from cross-modal feature retention, particularly in scenarios of extreme data imbalance. While prior research has investigated methods for tackling long-tailed imbalances in multimodal learning, leveraging techniques such as LMPT and deep generative modeling, these often depend on hand-engineered features or struggle with robust representation learning. We show that our approach not only consistently improves robustness and generalization capabilities through a unified modality fusion mechanism but also performs well in unsupervised anomaly detection, distinguishing itself from existing models that frequently suffer from noise sensitivity and struggle to maintain performance across varied datasets.
